# Redes Neuronales
Josue Valenzuela

### Vamos a importar la data con la utilidad prporcionada por [ zalandoresearch ](https://github.com/zalandoresearch/fashion-mnist)

In [1]:
from utils import mnist_reader

Vamos a comenzar por el set de entrenamiento

In [2]:
X_train, y_train = mnist_reader.load_mnist('data', kind='train')

Continuamos con los datos de prueba

In [3]:
X_test, y_test = mnist_reader.load_mnist('data', kind='t10k')

### Vamos a transformar los datos para poder trabajar con ellos


Vamos a comenzar convirtiendo a int64 los datos para facilitar el manejo

In [4]:
import numpy as np
X_train = np.array(X_train, dtype=np.int64)
X_test = np.array(X_test, dtype=np.int64)

Vamos a normalizar los datos

In [5]:
X_train, X_test = (X_train / 1000.0, X_test / 1000.0)

Vamos a transformar los datos de label, a una forma mas apropiada

In [6]:
m_train, n_train = X_train.shape
m_test, n_test = X_test.shape

In [8]:
y_train = y_train.reshape(m_train, 1)
y_test = y_test.reshape(m_test, 1)

### Arquitectura de la red neuronal

Vamos a comenzar con:
- **Capa de entrada:** 784 neuronas, ya que es el numero de pixeles de la imagen
- ** Capa escondida:** 125 neuronas
- **Capa de salida:** 10 neuronas ya que es el total de prendas posibles, se detallan a continuacion:


| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

In [9]:
NN_ARCH = np.array( [ n_train, 125, 10 ] )

## Optimizacion de los datos

In [10]:
from utils.neural_network import *

Creamos la matriz de theta en base a la arquitectura del modelo

In [11]:
theta_shapes  = np.hstack((
    NN_ARCH[1:].reshape(len(NN_ARCH) - 1, 1),
    (NN_ARCH[:-1] + 1).reshape(len(NN_ARCH) - 1, 1)
))

Vamos a comenzar con valores de theta aleatorios

In [12]:
flat_thetas = flatten_list_of_arrays([
    np.random.rand(*theta_shape)
    for theta_shape in theta_shapes
])

Comenzamos con la optimizacion

In [13]:
from scipy import optimize as op

In [14]:
Y = (y_train == np.array(range(10))).astype(int)

In [ ]:
opt_res = op.minimize(
    fun=cost_function,
    x0=flat_thetas,
    args=(theta_shapes, X_train, Y),
    method='L-BFGS-B',
    jac=back_propagation,
    options={'disp': True, 'maxiter': 3000}
)

/home/davalenzuela/Documents/uvg/ai/neural-network/fashion_neural_network/utils/neural_network.py:60: RuntimeWarning: divide by zero encountered in log
  return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)
/home/davalenzuela/Documents/uvg/ai/neural-network/fashion_neural_network/utils/neural_network.py:60: RuntimeWarning: invalid value encountered in multiply
  return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)


Vamos a guardar el modelo entrenado para evitar correrlo de nuevo

In [0]:
import pickle
with open('pre-trained/pre-trained-model-v1', 'wb') as o_file:
  pickle.dump(opt_res.x, o_file)
  o_file.close()

## Veamos la efectividad del modelo

Vamos a leer los datos del modelo pre entrenado

In [ ]:
with open('pre-trained-model-v1', 'rb') as o_file:
    opt_thetas = pickle.load(o_file)
    o_file.close()

### Vamos a predecir los valores en base al modelo encontrado

In [ ]:
thetas = inflate_matrixes(
    opt_thetas,
    theta_shapes
)

Utilizamos feed forward

In [ ]:
ff_res = feed_forward(
    thetas,
    X_test
)

Vamos a medir la precision del modelo

In [ ]:
predict_lbl = np.argmax(ff_res[-1], axis = 1)

In [ ]:
prediction_res = {'correct': 0, 'incorrect': 0}
for i in range( len( ff_res[-1] ) ):
    if(predict_lbl[i] == y_test[i][0]):
        prediction_res['correct'] += 1
    else:
        prediction_res['incorrect'] +=1

print(prediction_res)
print(f"Precisión: { round( ( prediction_res['correct'] / len( ff_res[-1] ) ) * 100 ) }%")